# 1 Building LLM Apps

In [1]:
# !pip install langchain_google_genai

In [4]:
# Simple Chatbot

# 1 Imports
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# load model
api = "api here"
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash',google_api_key=api)

# UI

while True:
  query = input("Enter Query:  ")
  if query == "exit":
    print("Good buy \n")
    break
  response = llm.invoke(query).content
  print("\n\n")
  print(response)

Enter Query:  how are you



I am doing well, thank you for asking!  How are you today?
Enter Query:  what is ml



ML stands for **Machine Learning**.  It's a branch of artificial intelligence (AI) and computer science which focuses on the use of data and algorithms to imitate the way that humans learn, gradually improving its accuracy.  Instead of being explicitly programmed, machine learning systems learn from data.
Enter Query:  thanks



You're welcome! Is there anything else I can help you with?
Enter Query:  exit
Good buy 



# 2 Building LLMs Models

Step-by-step outline:

Prepare a small labeled dataset (e.g., comments with sentiment labels or categories).

Load a small pre-trained transformer model (e.g., distilbert-base-uncased).

Tokenize the dataset.

Fine-tune the model on the dataset.

Evaluate immediately after training with metrics like accuracy or classification report.

In [5]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

data = {
    "text": [
        "This is great!",
        "I hate this.",
        "Not bad at all.",
        "Could be better.",
        "Absolutely fantastic!",
        "Worst experience ever.",
    ],
    "label": [1, 0, 1, 0, 1, 0],
}

# 2. Convert to Dataset
dataset = Dataset.from_dict(data)

dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 6
})

In [7]:
# 3. Load tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# 4. Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 5. Define train/test split (here small so just use all for train and eval)
train_dataset = tokenized_dataset
eval_dataset = tokenized_dataset

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [10]:
# 6. Define compute metrics function
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# 7. Training arguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    # You can add other parameters as needed
)


# 8. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# 9. Train & Evaluate
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 611noorsaeed (noorsaeed611) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=3, training_loss=0.6545107762018839, metrics={'train_runtime': 224.2389, 'train_samples_per_second': 0.08, 'train_steps_per_second': 0.013, 'total_flos': 2384413175808.0, 'train_loss': 0.6545107762018839, 'epoch': 3.0})

In [11]:
eval_results = trainer.evaluate()

print("Evaluation results:", eval_results)

Evaluation results: {'eval_loss': 0.5919930338859558, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 8.417, 'eval_samples_per_second': 0.713, 'eval_steps_per_second': 0.119, 'epoch': 3.0}
